# Develop, Document, Demonstrate

## zplain.py module

In [1]:
import os
import sys

SRC_DIR = '../scalygraphic'
sys.path.insert(1, SRC_DIR)

os.listdir(SRC_DIR)

['zplain.py']

In [ ]:
# %%writefile ../src/zplain.py
# %load ../scalygraphic/zplain.py
"""
complex plane under a grid of pixels
with printing interface
"""
import numpy as np

DEFAULT_PLAIN = {'center_point': 0 + 0j,
                 'zoom': 1/2,
                 'theta': 0,
                 'n_rows': 32,
                 'n_cols': 32}

def get_default_frame():
    """  """
    def_dict = DEFAULT_PLAIN
    complex_frame = get_frame_from_dict(DEFAULT_PLAIN)
    default_frame = {**complex_frame, **def_dict}
    
    return default_frame


def get_frame_from_dict(def_dict=None):
    """ complex_frame, def_dict = get_frame_from_dict(def_dict)
        legacy wrapper function.
    Args:
        def_dict: definition dictionary with keys:
                    'center_point', 'zoom', 'theta', 'n_rows', 'n_cols'
    Returns:
        complex_frame:
        def_dict:
    """
    if def_dict is None or not isinstance(def_dict, dict):
        def_dict = DEFAULT_PLAIN
        
    complex_frame = get_complex_frame(
        def_dict['center_point'],
        def_dict['zoom'],
        def_dict['theta'],
        def_dict['n_rows'],
        def_dict['n_cols'])

    return complex_frame


def get_complex_frame(CP, ZM, theta, h=1, w=1):
    """ get the complex numbers at ends and centers of a frame  """
    frame_dict = {'center_point':CP}
    if w >= h:
        frame_dict['top_center'] = np.exp(1j*(np.pi/2 + theta))/ZM
        frame_dict['right_center'] = (w/h) * np.exp(1j * theta) / ZM
    else:
        frame_dict['top_center'] = (h/w) * np.exp(1j*(np.pi/2 + theta)) / ZM
        frame_dict['right_center'] = np.exp(1j * theta) / ZM

    frame_dict['bottom_center'] = frame_dict['top_center'] * -1
    frame_dict['left_center'] = frame_dict['right_center'] * -1
    frame_dict['upper_right'] = frame_dict['right_center'] + frame_dict['top_center']
    frame_dict['bottom_right'] = frame_dict['right_center'] + frame_dict['bottom_center']

    frame_dict['upper_left'] = frame_dict['left_center'] + frame_dict['top_center']
    frame_dict['bottom_left'] = frame_dict['left_center'] + frame_dict['bottom_center']

    for k in frame_dict.keys():
        frame_dict[k] = frame_dict[k] + CP

    return frame_dict


def get_complex_pixel_matrix(frame_dict):
    """ complex_pixel_matrix = get_complex_pixel_matrix(frame_dict) 
    Args:
        frame_dict:             as defined herein
        
    Returns:
        complex_pixel_matrix:   matrix of pixels over points on the complex plain defined by frame_dict
    """
    n_rows = frame_dict['n_rows']
    n_cols = frame_dict['n_cols']
    
    complex_pixel_matrix = np.zeros((n_rows, n_cols), dtype=complex)
    left_style = np.linspace(frame_dict['upper_left'], frame_dict['bottom_left'], frame_dict['n_rows'])
    right_style = np.linspace(frame_dict['upper_right'], frame_dict['bottom_right'], frame_dict['n_rows'])
    for row_number in range(n_rows):
        complex_pixel_matrix[row_number,:] = np.linspace(left_style[row_number], 
                                                         right_style[row_number], 
                                                         n_cols)
    
    return complex_pixel_matrix


def complex_to_string(Z, n_dec=3):
    """ Z_string = complex_to_string(Z, N_DEC)
    complex number to string with decimal places N_DEC 
    Args:
        Z:          a complex number
        n_dec:      number of decimal places for output
    Returns:
        Z_str:      the complex number as a string
    """
    fmt = '.{}f'.format(n_dec)
    cfmt = "{z.real:" + fmt + "}{z.imag:+" + fmt + "}j"

    return cfmt.format(z=Z)


def show_complex_matrix(Z0,N_DEC=3):
    """ display a complex matrix or array """
    SPC = ' ' * 2
    if Z0.shape[0] == Z0.size:
        row_str = ''
        for col in range(0, Z0.shape[0]):
            row_str += complex_to_string(Z0[col], N_DEC) + SPC + '\n'
        print(row_str)
   
    else:
        for row in range(0,Z0.shape[0]):
            row_str = ''
            for col in range(0, Z0.shape[1]):
                row_str += complex_to_string(Z0[row, col], N_DEC) + SPC

            print(row_str)

            
def rnd_lambda(s=1):
    """ rnd_lambda, lambda_dict = rnd_lambda(s=1)
    random parameters s.t. a*d - b*c = s """
    b = np.random.random()
    c = np.random.random()
    ad = b*c + 1
    a = np.random.random()
    d = ad / a
    [a, b, c, d] = [a, b, c, d] * s
    abcd_dict = {'a': a, 'b': b, 'c': c, 'd': d}
    abcd_set = np.array([a, b, c, d])
    
    return abcd_set, abcd_dict


def complex_frame_dict_to_string(frame_dict, N_DEC=4):
    """ get a formatted list of strings """
    
    print_order = ['upper_left', 'top_center', 'upper_right',
                   'left_center', 'center_point', 'right_center',
                  'bottom_left', 'bottom_center', 'bottom_right']

    STR_L = 14
    frame_string = ''
    row = 0
    for k in print_order:
        z_str = complex_to_string(frame_dict[k], N_DEC)
        PAD = ' ' * (STR_L - len(z_str))
        frame_string += k + ':' + PAD + z_str
        row += 1
        if np.mod(row,3) == 0:
            frame_string += '\n'
        else:
            frame_string += '\t'
            
    return frame_string



In [3]:
frame_dict = get_default_frame()
# for k, v in frame_dict.items():
#     print(k, v, type(v))
c_p = get_complex_pixel_matrix(frame_dict)
print(c_p.shape)
show_complex_matrix(c_p, N_DEC=2)

(32, 32)
-2.00+2.00j  -1.87+2.00j  -1.74+2.00j  -1.61+2.00j  -1.48+2.00j  -1.35+2.00j  -1.23+2.00j  -1.10+2.00j  -0.97+2.00j  -0.84+2.00j  -0.71+2.00j  -0.58+2.00j  -0.45+2.00j  -0.32+2.00j  -0.19+2.00j  -0.06+2.00j  0.06+2.00j  0.19+2.00j  0.32+2.00j  0.45+2.00j  0.58+2.00j  0.71+2.00j  0.84+2.00j  0.97+2.00j  1.10+2.00j  1.23+2.00j  1.35+2.00j  1.48+2.00j  1.61+2.00j  1.74+2.00j  1.87+2.00j  2.00+2.00j  
-2.00+1.87j  -1.87+1.87j  -1.74+1.87j  -1.61+1.87j  -1.48+1.87j  -1.35+1.87j  -1.23+1.87j  -1.10+1.87j  -0.97+1.87j  -0.84+1.87j  -0.71+1.87j  -0.58+1.87j  -0.45+1.87j  -0.32+1.87j  -0.19+1.87j  -0.06+1.87j  0.06+1.87j  0.19+1.87j  0.32+1.87j  0.45+1.87j  0.58+1.87j  0.71+1.87j  0.84+1.87j  0.97+1.87j  1.10+1.87j  1.23+1.87j  1.35+1.87j  1.48+1.87j  1.61+1.87j  1.74+1.87j  1.87+1.87j  2.00+1.87j  
-2.00+1.74j  -1.87+1.74j  -1.74+1.74j  -1.61+1.74j  -1.48+1.74j  -1.35+1.74j  -1.23+1.74j  -1.10+1.74j  -0.97+1.74j  -0.84+1.74j  -0.71+1.74j  -0.58+1.74j  -0.45+1.74j  -0.32+1.74j  -0.19+1

In [4]:
#                                         Z_string = complex_to_string(Z, N_DEC)
Z = 1 + 1j
N_DEC = 3
Z_string = complex_to_string(Z, N_DEC)
print(Z_string)


1.000+1.000j


In [6]:

def complex_frame_dict_to_string(frame_dict, N_DEC=4):
    """ get a formatted list of strings """
    
    print_order = ['upper_left', 'top_center', 'upper_right',
                   'left_center', 'center_point', 'right_center',
                  'bottom_left', 'bottom_center', 'bottom_right']

    STR_L = 14
    frame_string = ''
    row = 0
    for k in print_order:
        z_str = complex_to_string(frame_dict[k], N_DEC)
        PAD = ' ' * (STR_L - len(z_str))
        frame_string += k + ':' + PAD + z_str
        row += 1
        if np.mod(row,3) == 0:
            frame_string += '\n'
        else:
            frame_string += '\t'
            
    return frame_string

default_frame = get_default_frame()

print(complex_frame_dict_to_string(default_frame, N_DEC=2))
# for k, v in default_frame.items():
#     print(k,v)
# S = get_aligned_frame_dict_string(default_frame)
# print(S)

upper_left:   -2.00+2.00j	top_center:    0.00+2.00j	upper_right:    2.00+2.00j
left_center:   -2.00+0.00j	center_point:    0.00+0.00j	right_center:    2.00+0.00j
bottom_left:   -2.00-2.00j	bottom_center:   -0.00-2.00j	bottom_right:    2.00-2.00j



In [7]:
rnd_lambda, lambda_dict = rnd_lambda(s=1)

for k,v in lambda_dict.items():
    print('%s ='%(k), v)
    
print(lambda_dict['a'] * lambda_dict['d'], lambda_dict['c'] * lambda_dict['b'])

a = 0.4127504192160484
b = 0.6401513254648472
c = 0.7145226844629777
d = 3.5309537572409324
1.457402643533676 0.457402643533676
